In [1]:
import os
os.chdir("../")

In [ ]:
from ngsolve import *
from netgen.geom2d import SplineGeometry

from netgen.occ import *
import netgen.meshing as ngm
from mpi4py.MPI import COMM_WORLD
import problems
import numpy as np
import params
import pickle
from time import time

In [ ]:
problem = problems.problem1

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G = problem[0]['G']
geom = problem[1]
BC = problem[2]
h = 1
ord = 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    geo = OCCGeometry(geom)
    mesh = Mesh(geo.GenerateMesh(maxh=h).Distribute(COMM_WORLD))
    return mesh
if COMM_WORLD.rank ==0:

    mesh = mesher(geom, h)
else:
    mesh = Mesh(ngm.Mesh.Receive(COMM_WORLD))

def F(u):
    return Id(3) + Grad(u)
def Norm(vec):
    return InnerProduct(vec, vec)**0.5

def Gel_energy_functional(F):
    gamma = G/KBTV
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    return 0.5*gamma* Trace(F.trans*F ) + H

def Gel_energy_EDP(F,v): ## |F|^2 + H => gamma F:Gradv + H'*J'
    # ddet(A(t))/dt = det(A(t))*trace(A^-1(t)*Grad (v))
    gamma = G/KBTV
    J = Det(F)
    phi = phi0/J
    dv = Grad(v)
    invF = Inv(F)
    H_prime = -phi/N + log(1-phi) + phi + chi*phi**2
    edp = gamma * InnerProduct(F,dv) + H_prime * J * Trace(invF*dv)
    return edp

def HW_EDP(F,P,v,q):
    """
     int Gamma F - PJF^-T:grad(v) = 0
     P - H'(J) = 0
    """
    Gamma = G/KBTV
    J = Det(F)
    phi = phi0/J
    dv = Grad(v)
    H_prime = -phi/N + log(1-phi) + phi + chi*phi**2
    invF = Inv(F)
    Int1 = Gamma*InnerProduct(F,dv) - P*J*Trace(invF*dv)
    Int2 = InnerProduct(-P - H_prime,q)


NameError: name 'COMM_WORLD' is not defined

In [3]:

## Generate spaces and forms
uSpace = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])
PSpace = L2(mesh, order=ord)
fes = FESpace([uSpace,PSpace])
u,P = fes.TrialFunction()
v,q = fes.TestFunction()
BF = BilinearForm(fes)
F = Id(3) + Grad(u)

## Assemble forms
def Assemble_Bilinear_Form(BF, F,v=None,P = None,q=None,form="Functional"):
    if form == "Functional":
        BF += Variation(Gel_energy_functional(F).Compile()*dx)
        return BF
    elif form == "EDP":
        BF += Gel_energy_EDP(F).Compile() * dx
        return BF
    elif form == "HW":
        BF += HW_EDP(F,P,v,q) * dx
        return BF

BF = Assemble_Bilinear_Form(BF, F,v,P,q, form)



NameError: name 'mesh' is not defined

In [ ]:
def Solver_freeswell(BF, gfu, tol=1e-8, maxiter=250, damp = 0.5):
    """
    Solves the problem
    """
    res = gfu.vec.CreateVector()
    w = gfu.vec.CreateVector()
    history = GridFunction(fes, multidim = 0)
    # here we may need to add another loop
   
    for iter in range(maxiter):
        # Prints before the iteration: number of it, residual, energy
        print("Energy: ", BF.Energy(gfu.vec), "Residual: ", sqrt(abs(InnerProduct(res,res))), "Iteration: ", iter)
        BF.Apply(gfu.vec, res)
        BF.AssembleLinearization(gfu.vec)
        inv = BF.mat.Inverse(freedofs = fes.FreeDofs())        
        w.data = damp * inv * res
        gfu.vec.data -= w
        history.AddMultiDimComponent(gfu.vec)
        if sqrt(abs(InnerProduct(w,res))) < tol:
            print("Converged")
            break
    return gfu, history

def petsc_solver(fes, BL, gfu):
    solver = NonLinearSolver(fes = fes,a = BF,solverParameters={"snes_type": "qn",
                                            "snes_max_it": 2000,
                                            "snes_monitor": "",
                                            "snes_rtol": 1e-6,
                                            "snes_linesearch_type": "basic",
                                            "snes_linesearch_damping": 0.3,
                                            "snes_linesearch_max_it": 100})
    gfu_petsc =solver.solve(gfu)
    return gfu_petsc

gfu = GridFunction(fes)
gfu.vec[:] = 0
#t1 =  time()
#gfu, history = Solver_freeswell(BF, gfu)
#print("Time on ngsolve newton:", abs(t1-time()))

#gfu = GridFunction(fes)
#gfu.vec[:] = 0
t1 = time() 
gfu = petsc_solver(fes,BF, gfu)
print("Time on snes newton:", abs(t1-time()))